### Used the Galaxy manipulation tool `Filter sequences by ID from a tabular file`to get just the _T.wesiss_ protein sequences that are 1) found on Day 0 and 2) from the cytoplasm.

### PROTEUS2 protein secondary structure analysis of remanent (survivor) proteins:

### This is a notebook to combine and evaluate the protein secondary structure output for Day 0 cytoplasm protein sequences.

### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/database-fastas/

/home/millieginty/Documents/git-repos/rot-mayer/data/database-fastas


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

### Galaxy wasn't letting me download the filtered fasta (see above), so I did so manually for the Day 0 DB diatom cytoplasm proteins. Called `T0-322-trypsin-DB-diatom-cytoplasm.fasta`.

### Need to change all the `I` residues to `L` in the original fasta so I can manually search for protein IDs using the UniPept LCA peptides (which don't have `I`'s). 

### Still have to remove any `X` residues, though, because Proteus2 will not accept sequences with them. Using `sed`:

In [3]:
ls

mmetsp_uniprot_tw_combo.fasta  mmetsp_uniprot_tw_combo_singleline.fasta


In [5]:
# use sed to find all I's and replace with L's

!sed 's/I/L/g' mmetsp_uniprot_tw_combo_singleline.fasta > mmetsp_uniprot_tw_combo_singleline_IL.fasta

In [6]:
# use sed to remove all occurances of X

!sed 's/X//g' mmetsp_uniprot_tw_combo_singleline_IL.fasta > mmetsp_uniprot_tw_combo_singleline_IL_noX.fasta

### Normally create the list of _T. weiss_ protein IDs wanted to pull out of the larger TW database using the Galaxy `Filter sequences by ID from a tabular file` tool. The list I get by searching the UniPept LCA peptides in the IL-adjusted database. 

### Still, since there are few proteins here I can just do it manually, pulling out the desired protein sequences into a new FASTA to submit to Proteus2.

### This worked! Proteus2 accepted the submission. Now the results come in an email from Proteus2 in a garbage format:

### The output of Proteus2 comes in an email in nominally FASTA format, but with extra line breaks and spacings within protein and prediction sequences. Also, the name gets cut off.

#### They look like this:

#### >Thalas

MMKLAALAAL MGSAAAFAPA QTGKASTQLR AFEDELGAQP PLGFFDPFGM 

CCHHHHHHHH HHHHHCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

LSGDCTQERF DRLRYVEIKH GRICMLAFLG QIVTRAGIHL PGSINYAGDS 

CCCCCCHHHH HHHHHHHHHH HHHHHHHHHH HHHHHHHCCC CCCCCCCCCC 

FDSFPNGVAA LFGPNSIPTA GLVQIIAFIG VLECAFMRDV PGTGNEFVGD 

CCCCCCCCCC CCCCCCCCHH HHHHHHHHHH HHHHHHHHCC CCCCCCCCCC 

FRNGYIDFGW DDFDEETKLQ KRAIQSGTIS NMMKLAALAA LMGSAAAFAP 

CCCCCCCCCC CCCCHHHHHH HHHHHHHHHH HHHHHHHHHC CCCCCCCCCC 



### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [8]:
cd /home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/

/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2


In [9]:
!head T0-322-trypsin-DB-diatom-cytoplasm-proteins

>Thalas

MKFLAALSFA ALNGAAAFTA TGSNRATTRL SMSDYDANAS VRDMAAANLG 
CCHHHHHHHH CCCCEEEEEE ECCCCEEEEE EECCCCCCCC CHHHHHCCCC 

VGSGGSPLYK TGGTARDLAE LWDNSSPVTE GDVKRESALN LNTHSSLLVL 
CCCCCCCCCC CCCCHHHHHH CCCCCCCCCC CCCCCCCCCC CCCCEEEEEE 

EHKHPKTRHL LVSLTYQHAS EKLLKHTSTM KFSAALALAA LNGAAAFVAP 
ECCCCCCEEE EEEEEEHHHH HHHHHHCCCE EEEEEEEEEE ECCCEEEECC 


In [10]:
# remove empty lines
# remove empty spaces from lines
# remove carat protein sequence names
# collapse the 2 lines from every protein sequence line (protein seq, secondard stucture pred.) into one line

!sed '/^[[:space:]]*$/d' T0-322-trypsin-DB-diatom-cytoplasm-proteins | cat \
| sed '/>/d' \
| tr -d "[:blank:]" > T0-322-trypsin-DB-diatom-cytoplasm-proteins.txt

!awk '{printf "%s%s",$0,(NR%2?FS:RS)}' T0-322-trypsin-DB-diatom-cytoplasm-proteins.txt > \
T0-322-trypsin-DB-diatom-cytoplasm-proteins_sort.csv

In [11]:
!head T0-322-trypsin-DB-diatom-cytoplasm-proteins_sort.csv

MKFLAALSFAALNGAAAFTATGSNRATTRLSMSDYDANASVRDMAAANLG CCHHHHHHHHCCCCEEEEEEECCCCEEEEEEECCCCCCCCCHHHHHCCCC
VGSGGSPLYKTGGTARDLAELWDNSSPVTEGDVKRESALNLNTHSSLLVL CCCCCCCCCCCCCCHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCEEEEEE
EHKHPKTRHLLVSLTYQHASEKLLKHTSTMKFSAALALAALNGAAAFVAP ECCCCCCEEEEEEEEEHHHHHHHHHHCCCEEEEEEEEEEEECCCEEEECC
KGPVASTQLAMSDYRTSVRDDLGVGSGGSPLYKTGGTARDLAELWDNSSP CCCCCCCEECCHHHHHHHHHCCCCCCCCCCCCCCCCCHHHHHHHCCCCCE
VLVQGGSLRTWSFANPALESVQVLLKTEGRPLDADVELWQGPDNTPHKMR EEEECCCCEEEECCCCHHHHHHHHHCCCCCCCCCCCCEECCCCCCCEEEE
VYVEDGALRTFNAVLGTPRGPNTVALRNLGQLEFPLDAVVRPDRDDGLAA EEEECCCCEEECCCCCCCCCCEEEEEECCCCCCCCCCCCCCCCCCCCHHH
GLASVATRSETLQGGALRTYPFNPDLAELWDNSSPVLVQGGSLRTWSFAN HHHHHHHHHHCCCCCEEEECCCCCCCCCCCCCCCEEEEECCCCEEECCCC
PALDSVQVLLKTEGRPLDCDLELWQGPDNTPHKMRVYVEDGALRTFSAVL CCCCCEEEEEECCCCCCCCCCCCCCCCCCCCEEEEEEEECCCCEECCCCC
GTPRGPNTLAVRNLGQLEFPLDAVVRPDRGDGLAQSLAAVSTLSETLQGG CCCCCCEEEEEECCCCCCCCCCCECCCCCCCHHHHHHHHHHHHHHHHCCC
ALRTYPFSAAVDSVAVLLKTDGRPLNARLELLQGPNNNKQVVELYTEDGL CEEECCCCCCCCEEEEEEEECCCCCCCCHHE

In [12]:
# read into pandas with space delimeter

PeaksDB_322_cyt_prot = pd.read_csv("T0-322-trypsin-DB-diatom-cytoplasm-proteins_sort.csv", delim_whitespace=True, header=None)


# name columns

PeaksDB_322_cyt_prot.columns =['Stripped protein sequence', 'Secondary structure pred.'] 

In [13]:
PeaksDB_322_cyt_prot.head(13)

,Stripped protein sequence,Secondary structure pred.
0,MKFLAALSFAALNGAAAFTATGSNRATTRLSMSDYDANASVRDMAA...,CCHHHHHHHHCCCCEEEEEEECCCCEEEEEEECCCCCCCCCHHHHH...
1,VGSGGSPLYKTGGTARDLAELWDNSSPVTEGDVKRESALNLNTHSS...,CCCCCCCCCCCCCCHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCEE...
2,EHKHPKTRHLLVSLTYQHASEKLLKHTSTMKFSAALALAALNGAAA...,ECCCCCCEEEEEEEEEHHHHHHHHHHCCCEEEEEEEEEEEECCCEE...
3,KGPVASTQLAMSDYRTSVRDDLGVGSGGSPLYKTGGTARDLAELWD...,CCCCCCCEECCHHHHHHHHHCCCCCCCCCCCCCCCCCHHHHHHHCC...
4,VLVQGGSLRTWSFANPALESVQVLLKTEGRPLDADVELWQGPDNTP...,EEEECCCCEEEECCCCHHHHHHHHHCCCCCCCCCCCCEECCCCCCC...
5,VYVEDGALRTFNAVLGTPRGPNTVALRNLGQLEFPLDAVVRPDRDD...,EEEECCCCEEECCCCCCCCCCEEEEEECCCCCCCCCCCCCCCCCCC...
6,GLASVATRSETLQGGALRTYPFNPDLAELWDNSSPVLVQGGSLRTW...,HHHHHHHHHHCCCCCEEEECCCCCCCCCCCCCCCEEEEECCCCEEE...
7,PALDSVQVLLKTEGRPLDCDLELWQGPDNTPHKMRVYVEDGALRTF...,CCCCCEEEEEECCCCCCCCCCCCCCCCCCCCEEEEEEEECCCCEEC...
8,GTPRGPNTLAVRNLGQLEFPLDAVVRPDRGDGLAQSLAAVSTLSET...,CCCCCCEEEEEECCCCCCCCCCCECCCCCCCHHHHHHHHHHHHHHH...
9,ALRTYPFSAAVDSVAVLLKTDGRPLNARLELLQGPNNNKQVVELYT...,CEEECCCCCCCCEEEEEEEECCCCCCCCHHEEECCCCCEEEEEEEE...


In [18]:
# add a column with the stripped peptide length (number of AAs)
PeaksDB_322_cyt_prot['Sequence length'] = PeaksDB_322_cyt_prot['Stripped protein sequence'].apply(len)

# use a count function to enumerate the # of C's (coil residues) for each peptide
PeaksDB_322_cyt_prot['C'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("C")

# use a count function to enumerate the # of H's (helices residues) in each peptide
PeaksDB_322_cyt_prot['H'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("H")

# use a count function to enumerate the # of E's (beta strand residues) in each peptide
PeaksDB_322_cyt_prot['E'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("E")

#use a count function to enumerate the # of T's (membrane helix residues) in each peptide
PeaksDB_322_cyt_prot['T'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("T")

# use a count function to enumerate the # of B's (membrane strand residues) in each peptide
PeaksDB_322_cyt_prot['B'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("B")

# use a count function to enumerate the # of B's (signal peptide residues) in each peptide
PeaksDB_322_cyt_prot['S'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("S")

# use a count function to enumerate the # of c's (cleavage site residues) in each peptide
PeaksDB_322_cyt_prot['c'] = PeaksDB_322_cyt_prot['Secondary structure pred.'].str.count("c")

# add a column with the % C
PeaksDB_322_cyt_prot['% C'] = PeaksDB_322_cyt_prot['C'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % H
PeaksDB_322_cyt_prot['% H'] = PeaksDB_322_cyt_prot['H'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % E
PeaksDB_322_cyt_prot['% E'] = PeaksDB_322_cyt_prot['E'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % T
PeaksDB_322_cyt_prot['% T'] = PeaksDB_322_cyt_prot['T'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % B
PeaksDB_322_cyt_prot['% B'] = PeaksDB_322_cyt_prot['B'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % S
PeaksDB_322_cyt_prot['% S'] = PeaksDB_322_cyt_prot['S'] / PeaksDB_322_cyt_prot['Sequence length']

# add a column with the % c
PeaksDB_322_cyt_prot['% c'] = PeaksDB_322_cyt_prot['c'] / PeaksDB_322_cyt_prot['Sequence length']

# additive check

PeaksDB_322_cyt_prot['% check'] = PeaksDB_322_cyt_prot['% C'] + PeaksDB_322_cyt_prot['% H'] + PeaksDB_322_cyt_prot['% E'] \
                                + PeaksDB_322_cyt_prot['% T'] + PeaksDB_322_cyt_prot['% B'] + \
                                PeaksDB_322_cyt_prot['% S'] + PeaksDB_322_cyt_prot['% c']

In [19]:
PeaksDB_322_cyt_prot.head()

,Stripped protein sequence,Secondary structure pred.,Sequence length,C,H,E,T,B,S,c,% C,% H,% E,% T,% B,% S,% c,% check
0,MKFLAALSFAALNGAAAFTATGSNRATTRLSMSDYDANASVRDMAA...,CCHHHHHHHHCCCCEEEEEEECCCCEEEEEEECCCCCCCCCHHHHH...,50,23,13,14,0,0,0,0,0.46,0.26,0.28,0.0,0.0,0.0,0.0,1.0
1,VGSGGSPLYKTGGTARDLAELWDNSSPVTEGDVKRESALNLNTHSS...,CCCCCCCCCCCCCCHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCEE...,50,38,6,6,0,0,0,0,0.76,0.12,0.12,0.0,0.0,0.0,0.0,1.0
2,EHKHPKTRHLLVSLTYQHASEKLLKHTSTMKFSAALALAALNGAAA...,ECCCCCCEEEEEEEEEHHHHHHHHHHCCCEEEEEEEEEEEECCCEE...,50,14,10,26,0,0,0,0,0.28,0.20,0.52,0.0,0.0,0.0,0.0,1.0
3,KGPVASTQLAMSDYRTSVRDDLGVGSGGSPLYKTGGTARDLAELWD...,CCCCCCCEECCHHHHHHHHHCCCCCCCCCCCCCCCCCHHHHHHHCC...,50,31,16,3,0,0,0,0,0.62,0.32,0.06,0.0,0.0,0.0,0.0,1.0
4,VLVQGGSLRTWSFANPALESVQVLLKTEGRPLDADVELWQGPDNTP...,EEEECCCCEEEECCCCHHHHHHHHHCCCCCCCCCCCCEECCCCCCC...,50,27,9,14,0,0,0,0,0.54,0.18,0.28,0.0,0.0,0.0,0.0,1.0


In [20]:
index = ['322 total']

data = {
        '% C total': PeaksDB_322_cyt_prot['% C'].sum(),
        '% H total': PeaksDB_322_cyt_prot['% H'].sum(),
        '% E total': PeaksDB_322_cyt_prot['% E'].sum(),
        '% T total': PeaksDB_322_cyt_prot['% T'].sum(),
        '% B total': PeaksDB_322_cyt_prot['% B'].sum(),
        '% S total': PeaksDB_322_cyt_prot['% S'].sum(),
        '% c total': PeaksDB_322_cyt_prot['% c'].sum(),
        '% check sum': PeaksDB_322_cyt_prot['% check'].sum()
       }

PeaksDB_322_cyt_prot_totals = pd.DataFrame(data, columns=['% C total', '% H total', '% E total', '% T total', \
                                                      '% B total',  '% S total', '% c total', \
                                                      '% check sum'], index=index)

PeaksDB_322_cyt_prot_totals['overall % sum'] = PeaksDB_322_cyt_prot_totals['% C total'] \
                                            + PeaksDB_322_cyt_prot_totals['% H total'] \
                                            + PeaksDB_322_cyt_prot_totals['% E total'] \
                                            + PeaksDB_322_cyt_prot_totals['% T total'] \
                                            + PeaksDB_322_cyt_prot_totals['% B total'] \
                                            + PeaksDB_322_cyt_prot_totals['% S total'] \
                                            + PeaksDB_322_cyt_prot_totals['% c total'] 


PeaksDB_322_cyt_prot_totals['overall % C'] = PeaksDB_322_cyt_prot_totals['% C total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % H'] = PeaksDB_322_cyt_prot_totals['% H total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % E'] = PeaksDB_322_cyt_prot_totals['% E total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % T'] = PeaksDB_322_cyt_prot_totals['% T total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % B'] = PeaksDB_322_cyt_prot_totals['% B total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % S'] = PeaksDB_322_cyt_prot_totals['% S total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

PeaksDB_322_cyt_prot_totals['overall % c'] = PeaksDB_322_cyt_prot_totals['% c total'] / PeaksDB_322_cyt_prot_totals['overall % sum']

# write to csv

PeaksDB_322_cyt_prot_totals.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/Day0_322 PeaksDB_dia_cyt_prot_trypsin_totals")

PeaksDB_322_cyt_prot_totals.head()

,% C total,% H total,% E total,% T total,% B total,% S total,% c total,% check sum,overall % sum,overall % C,overall % H,overall % E,overall % T,overall % B,overall % S,overall % c
322 total,66.155875,32.609961,37.034164,1.2,0.0,0.0,0.0,137.0,137.0,0.48289,0.238029,0.270322,0.008759,0.0,0.0,0.0


In [ ]:
# Now for the membrane proteins

In [21]:
!head T0-322-trypsin-DB-diatom-membrane-proteins

>tr|A0A

MALSSTERRS KNVQVFVEKD AVETSFAKWA QPGHFSRTLA KGPKTTTWLW 
CCCCCCCCCC CCCCEEEEEC CCCCCHHHHH CCCCCCCCCC CCCEEEHHHH 

NLHADAHDFD SQTNSLEEVS RKLFSAHFGQ LALLFLWLSG MHFHGAYFSN 
HHHHHCCCCC CCCCCTTTTT TTTTTTTTTT TTTTTTTTTT TTTTTTTCCH 

YSAWLTDPLS LKQSSQVVWP LVGQELLNAD VGGNFQGVQT TSGWFQMWRA 
HHHHHHCCCC EEEECCCCCC CCCHHHHCCC CCCCEEEEEE CCCHHHHHHH 


In [22]:
# remove empty lines
# remove empty spaces from lines
# remove carat protein sequence names
# collapse the 2 lines from every protein sequence line (protein seq, secondard stucture pred.) into one line

!sed '/^[[:space:]]*$/d' T0-322-trypsin-DB-diatom-membrane-proteins | cat \
| sed '/>/d' \
| tr -d "[:blank:]" > T0-322-trypsin-DB-diatom-membrane-proteins.txt

!awk '{printf "%s%s",$0,(NR%2?FS:RS)}' T0-322-trypsin-DB-diatom-membrane-proteins.txt > \
T0-322-trypsin-DB-diatom-membrane-proteins_sort.csv

In [23]:
!head T0-322-trypsin-DB-diatom-membrane-proteins_sort.csv

MALSSTERRSKNVQVFVEKDAVETSFAKWAQPGHFSRTLAKGPKTTTWLW CCCCCCCCCCCCCCEEEEECCCCCCHHHHHCCCCCCCCCCCCCEEEHHHH
NLHADAHDFDSQTNSLEEVSRKLFSAHFGQLALLFLWLSGMHFHGAYFSN HHHHHCCCCCCCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCH
YSAWLTDPLSLKQSSQVVWPLVGQELLNADVGGNFQGVQTTSGWFQMWRA HHHHHHCCCCEEEECCCCCCCCCHHHHCCCCCCCEEEEEECCCHHHHHHH
EGLTSEVELYWTALGGLAMSALMLFAGWFHYHKAAPKLEWFQNAESMMNH HCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTCEEEEHHHHCTTTTTTTT
HLAGLLGLGCLSWSGHQLHLALPLNKLLDAGVSPQELPLPHEFLLNRDLM TTTTTTTTTTTTTTTTTTTCHHHHHHHHHHCCCCCCCCCHHHHHHHHHHH
AQLYPSFGKGLAPFFSGNWGEYSDFLTFKGGLNPVTGGLWLSDLAHHHLA HHCCCCCCCCCCCCCCCHHHHHHHCEEEECCCCCCCCEEETTTTTTTTTT
LAVLFLVAGHMYRTNWGLGHNMKELLEAHKGPFTGEGHKGLYELLTTSWH TTTTTTTTTTCEEECCCCCCCHHHHHHHHCCCCCCCCCCCHHHHHHCCTT
AQLALNLAMMGSLSLLVAHHMYAMPPYPYLATDYATQLSLFTHHMWLGGF TTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCCCTTTTTTTTTTTTTTTTT
CVVGGAAHGALFMVRDYTPANNYNNLLDRVLRHRDALLSHLNWVCLFLGC TTTTTTTTTTTTTTTEEECCCCCCCHHHHHHHCTTTTTTTTTTTTTTTTT
HAFGFYLHNDTMRALGRPQDMFSDKALQLQPLFAQWLQNLHLLAPGTTAP TTTTTTTTTTTTTTTCCCCCEEECCEEECCH

In [24]:
# read into pandas with space delimeter

PeaksDB_322_mem_prot = pd.read_csv("T0-322-trypsin-DB-diatom-membrane-proteins_sort.csv", delim_whitespace=True, header=None)


# name columns

PeaksDB_322_mem_prot.columns =['Stripped protein sequence', 'Secondary structure pred.'] 

In [25]:
PeaksDB_322_mem_prot.head(13)

,Stripped protein sequence,Secondary structure pred.
0,MALSSTERRSKNVQVFVEKDAVETSFAKWAQPGHFSRTLAKGPKTT...,CCCCCCCCCCCCCCEEEEECCCCCCHHHHHCCCCCCCCCCCCCEEE...
1,NLHADAHDFDSQTNSLEEVSRKLFSAHFGQLALLFLWLSGMHFHGA...,HHHHHCCCCCCCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
2,YSAWLTDPLSLKQSSQVVWPLVGQELLNADVGGNFQGVQTTSGWFQ...,HHHHHHCCCCEEEECCCCCCCCCHHHHCCCCCCCEEEEEECCCHHH...
3,EGLTSEVELYWTALGGLAMSALMLFAGWFHYHKAAPKLEWFQNAES...,HCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTCEEEEHHHHCTTTT...
4,HLAGLLGLGCLSWSGHQLHLALPLNKLLDAGVSPQELPLPHEFLLN...,TTTTTTTTTTTTTTTTTTTCHHHHHHHHHHCCCCCCCCCHHHHHHH...
5,AQLYPSFGKGLAPFFSGNWGEYSDFLTFKGGLNPVTGGLWLSDLAH...,HHCCCCCCCCCCCCCCCHHHHHHHCEEEECCCCCCCCEEETTTTTT...
6,LAVLFLVAGHMYRTNWGLGHNMKELLEAHKGPFTGEGHKGLYELLT...,TTTTTTTTTTCEEECCCCCCCHHHHHHHHCCCCCCCCCCCHHHHHH...
7,AQLALNLAMMGSLSLLVAHHMYAMPPYPYLATDYATQLSLFTHHMW...,TTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCCCTTTTTTTTTTTTT...
8,CVVGGAAHGALFMVRDYTPANNYNNLLDRVLRHRDALLSHLNWVCL...,TTTTTTTTTTTTTTTEEECCCCCCCHHHHHHHCTTTTTTTTTTTTT...
9,HAFGFYLHNDTMRALGRPQDMFSDKALQLQPLFAQWLQNLHLLAPG...,TTTTTTTTTTTTTTTCCCCCEEECCEEECCHHHHHHHHHHHHHHCC...


In [26]:
# add a column with the stripped peptide length (number of AAs)
PeaksDB_322_mem_prot['Sequence length'] = PeaksDB_322_mem_prot['Stripped protein sequence'].apply(len)

# use a count function to enumerate the # of C's (coil residues) for each peptide
PeaksDB_322_mem_prot['C'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("C")

# use a count function to enumerate the # of H's (helices residues) in each peptide
PeaksDB_322_mem_prot['H'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("H")

# use a count function to enumerate the # of E's (beta strand residues) in each peptide
PeaksDB_322_mem_prot['E'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("E")

#use a count function to enumerate the # of T's (membrane helix residues) in each peptide
PeaksDB_322_mem_prot['T'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("T")

# use a count function to enumerate the # of B's (membrane strand residues) in each peptide
PeaksDB_322_mem_prot['B'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("B")

# use a count function to enumerate the # of B's (signal peptide residues) in each peptide
PeaksDB_322_mem_prot['S'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("S")

# use a count function to enumerate the # of c's (cleavage site residues) in each peptide
PeaksDB_322_mem_prot['c'] = PeaksDB_322_mem_prot['Secondary structure pred.'].str.count("c")

# add a column with the % C
PeaksDB_322_mem_prot['% C'] = PeaksDB_322_mem_prot['C'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % H
PeaksDB_322_mem_prot['% H'] = PeaksDB_322_mem_prot['H'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % E
PeaksDB_322_mem_prot['% E'] = PeaksDB_322_mem_prot['E'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % T
PeaksDB_322_mem_prot['% T'] = PeaksDB_322_mem_prot['T'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % B
PeaksDB_322_mem_prot['% B'] = PeaksDB_322_mem_prot['B'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % S
PeaksDB_322_mem_prot['% S'] = PeaksDB_322_mem_prot['S'] / PeaksDB_322_mem_prot['Sequence length']

# add a column with the % c
PeaksDB_322_mem_prot['% c'] = PeaksDB_322_mem_prot['c'] / PeaksDB_322_mem_prot['Sequence length']

# additive check

PeaksDB_322_mem_prot['% check'] = PeaksDB_322_mem_prot['% C'] + PeaksDB_322_mem_prot['% H'] + PeaksDB_322_mem_prot['% E'] \
                                + PeaksDB_322_mem_prot['% T'] + PeaksDB_322_mem_prot['% B'] + \
                                PeaksDB_322_mem_prot['% S'] + PeaksDB_322_mem_prot['% c']

In [27]:
PeaksDB_322_mem_prot.head()

,Stripped protein sequence,Secondary structure pred.,Sequence length,C,H,E,T,B,S,c,% C,% H,% E,% T,% B,% S,% c,% check
0,MALSSTERRSKNVQVFVEKDAVETSFAKWAQPGHFSRTLAKGPKTT...,CCCCCCCCCCCCCCEEEEECCCCCCHHHHHCCCCCCCCCCCCCEEE...,50,33,9,8,0,0,0,0,0.66,0.18,0.16,0.00,0.0,0.0,0.0,1.0
1,NLHADAHDFDSQTNSLEEVSRKLFSAHFGQLALLFLWLSGMHFHGA...,HHHHHCCCCCCCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,50,12,6,0,32,0,0,0,0.24,0.12,0.00,0.64,0.0,0.0,0.0,1.0
2,YSAWLTDPLSLKQSSQVVWPLVGQELLNADVGGNFQGVQTTSGWFQ...,HHHHHHCCCCEEEECCCCCCCCCHHHHCCCCCCCEEEEEECCCHHH...,50,23,17,10,0,0,0,0,0.46,0.34,0.20,0.00,0.0,0.0,0.0,1.0
3,EGLTSEVELYWTALGGLAMSALMLFAGWFHYHKAAPKLEWFQNAES...,HCCCCTTTTTTTTTTTTTTTTTTTTTTTTTTTCEEEEHHHHCTTTT...,50,6,5,4,35,0,0,0,0.12,0.10,0.08,0.70,0.0,0.0,0.0,1.0
4,HLAGLLGLGCLSWSGHQLHLALPLNKLLDAGVSPQELPLPHEFLLN...,TTTTTTTTTTTTTTTTTTTCHHHHHHHHHHCCCCCCCCCHHHHHHH...,50,10,21,0,19,0,0,0,0.20,0.42,0.00,0.38,0.0,0.0,0.0,1.0


In [28]:
index = ['322 total']

data = {
        '% C total': PeaksDB_322_mem_prot['% C'].sum(),
        '% H total': PeaksDB_322_mem_prot['% H'].sum(),
        '% E total': PeaksDB_322_mem_prot['% E'].sum(),
        '% T total': PeaksDB_322_mem_prot['% T'].sum(),
        '% B total': PeaksDB_322_mem_prot['% B'].sum(),
        '% S total': PeaksDB_322_mem_prot['% S'].sum(),
        '% c total': PeaksDB_322_mem_prot['% c'].sum(),
        '% check sum': PeaksDB_322_mem_prot['% check'].sum()
       }

PeaksDB_322_mem_prot_totals = pd.DataFrame(data, columns=['% C total', '% H total', '% E total', '% T total', \
                                                      '% B total',  '% S total', '% c total', \
                                                      '% check sum'], index=index)

PeaksDB_322_mem_prot_totals['overall % sum'] = PeaksDB_322_mem_prot_totals['% C total'] \
                                            + PeaksDB_322_mem_prot_totals['% H total'] \
                                            + PeaksDB_322_mem_prot_totals['% E total'] \
                                            + PeaksDB_322_mem_prot_totals['% T total'] \
                                            + PeaksDB_322_mem_prot_totals['% B total'] \
                                            + PeaksDB_322_mem_prot_totals['% S total'] \
                                            + PeaksDB_322_mem_prot_totals['% c total'] 


PeaksDB_322_mem_prot_totals['overall % C'] = PeaksDB_322_mem_prot_totals['% C total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % H'] = PeaksDB_322_mem_prot_totals['% H total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % E'] = PeaksDB_322_mem_prot_totals['% E total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % T'] = PeaksDB_322_mem_prot_totals['% T total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % B'] = PeaksDB_322_mem_prot_totals['% B total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % S'] = PeaksDB_322_mem_prot_totals['% S total'] / PeaksDB_322_mem_prot_totals['overall % sum']

PeaksDB_322_mem_prot_totals['overall % c'] = PeaksDB_322_mem_prot_totals['% c total'] / PeaksDB_322_mem_prot_totals['overall % sum']

# write to csv

PeaksDB_322_mem_prot_totals.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/Day0_322 PeaksDB_dia_mem_prot_trypsin_totals")

PeaksDB_322_mem_prot_totals.head()

,% C total,% H total,% E total,% T total,% B total,% S total,% c total,% check sum,overall % sum,overall % C,overall % H,overall % E,overall % T,overall % B,overall % S,overall % c
322 total,118.687713,75.555557,31.814074,34.942655,0.0,0.0,0.0,261.0,261.0,0.454742,0.289485,0.121893,0.13388,0.0,0.0,0.0
